In [3]:
import numpy as np
import pandas as pd
import re
import json
import time
import os

from IPython.display import display
from IPython.core.display import display, HTML
from tqdm.notebook import tqdm
from collections import Counter

import datetime

C:\Users\USER\AppData\Local\Temp\ipykernel_17384\2444131415.py:9: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


# Load Dic

In [4]:
df_cate = pd.read_excel(r"C:\Users\USER\Desktop\All\Project\23_VRP\Data\경기\all_cate.xlsx")
df_cate.head()

,Unnamed: 0,document,label,prob
0,0,NaN,2,0.9985
1,1,완도 명가 다시마(),8,0.9995
2,2,콜라개,2,0.9964
3,3,한우사골(국내산 팩),8,0.9994
4,4,누룽지 100t,8,0.9997


In [5]:
dic_1 = {1:"대용식(빵)",
2:"농산물",
3:"유제품",
4:"축산물",
5:"수산물",
6:"기타신선식품 / 냉장",
7:"신선냉동식품",
8:"가공식품 / 반찬",
9:"냉동식품 / 냉동과자",
10:"즉석/편의식품",
11:"과자류",
12:"주류",
13:"음료류",
14:"조미료",
15:"분말류 / 차,커피",
16:"통조림/병",
17:"유아식품",
18:"건강식품",
19:"기름",
20:"견과류",
0:"곡물"}



In [6]:
match_dic = {col["document"]:dic_1[col["label"]] for row,col in df_cate.iterrows()}

In [7]:
di_list = [2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022]
Path_ = r"C:\Users\USER\Desktop\All\Project\23_VRP\Data\경기\all\{}"

In [8]:
col_dic = {
    2012:[["일자","분류","물품명","수량","단가","금액"],
         ["일자","분류","물품명","수량","단위","금액"]],
    2013:[["일자","분류","물품명","수량","단가","금액"],
          ["일자","분류","물품명","수량","단위","금액"]],
    2014:[["일자","분류","물품명","수량","단가","금액"],
          ["일자","분류","물품명","수량","장부단가","금액"]],
    2015:[["일자","분류","물품명","수량","단가","금액"],
          ["일자","분류","물품명","수량","단위","금액"]],
    2016:[["일자","분류","물품명","수량","단가","금액"],
          ["일자","분류","물품명","수량","단위","금액"]],
    2017:[["일자","분류","물품명","수량","단가","금액"],
          ["일자","분류","물품명","수량","단위","금액"]],
    2018:[["일자","분류","물품명","수량","단가","금액"],
          ["일자","분류","물품명","수량","단위","금액"]],
    2019:[["일자","분류","물품명","수량","단가","금액"],
          ["일자","분류","물품명","수량","단위","금액"]],

    2020:[["일자","분류","물품명","수량","금액"],
          ["일자","분류","물품명","수량","금액","이용자"]],
    2021:[["일자","분류","물품명","수량","금액"],
          ["일자","이용자","분류","물품명","수량","금액"]],
    2022:[["일자","분류","물품명","수량","금액"],
          ["일자","이용자","분류","물품명","수량","금액"]]
    
}

In [9]:
eli_col = ["기부자","장부단가","개인/단체","이용자"]

In [10]:
def change_col(df_,col_dic,dic_,type_):
    if type_ == "supply":
        col_ori = list(df_.columns)
        col_after = col_dic[dic_][0]
        col_dic = {}
        for i in range(len(col_ori)):
            key_ = col_ori[i]
            value_ = col_after[i]
            col_dic[key_] = value_
        df_ = df_.rename(columns=col_dic)
    elif type_ == "demand":
        col_ori = list(df_.columns)
        col_after = col_dic[dic_][1]
        col_dic = {}
        for i in range(len(col_ori)):
            key_ = col_ori[i]
            value_ = col_after[i]
            col_dic[key_] = value_
        df_ = df_.rename(columns=col_dic)
    return df_

In [11]:
def removal_spaces(text_):
    return " ".join(text_.split())

In [12]:
def removal_1(test_,pattern):
    reg = re.compile(r"[0-9]+ *{}".format(pattern))
    text_list = reg.findall(test_)
    for i in text_list:
        test_ = test_.replace(i,"")
    return test_

In [13]:
def cleansing(all_identity):
    data_1 = [removal_spaces(i) for i in all_identity]
    data_1 = [x.replace("/","") for x in data_1]
    data_1 = [x.lower() for x in data_1]
    data_1 = [removal_1(x,"kg") for x in data_1]
    data_1 = [removal_1(x,"g") for x in data_1]
    data_1 = [removal_1(x,"ml") for x in data_1]
    data_1 = [removal_1(x,"pack") for x in data_1]
    data_1 = [removal_1(x,"pet") for x in data_1]
    data_1 = [removal_1(x,"ea") for x in data_1]
    return data_1


In [ ]:
error_file = []
for year_ in di_list:
    all_list = []
    print(year_)
    path_ = Path_.format(year_)
    file_list = [i for i in os.listdir(path_) if ".xlsx" in i ]
    tq = tqdm(file_list)
    df_sall = pd.DataFrame()
    df_dall = pd.DataFrame()
    for bank_p in tq:
        #print(file_)
        bank_ = bank_p.replace(".xlsx","")
        path_file = path_ + "\\" + bank_p
        try:
            df_ = pd.ExcelFile(path_file)
            df_s = pd.read_excel(path_file,sheet_name=df_.sheet_names[0])
            df_d = pd.read_excel(path_file,sheet_name=df_.sheet_names[1])

            for i in ["접수센터","인수센터","센터","기부자","지역","개인/단체"]:
                if i in df_s.columns:
                    df_s  = df_s.drop(i,axis=1)
                if i in df_d.columns:
                    df_d = df_d.drop(i,axis=1)



            df_s = change_col(df_s,col_dic,year_,"supply")
            df_d = change_col(df_d,col_dic,year_,"demand")


            for i in ["기부자","장부단가","개인/단체","이용자"]:
                if i in df_s.columns:
                    df_s  = df_s.drop(i,axis=1)
                if i in df_d.columns:
                    df_d = df_d.drop(i,axis=1)

            ## cleansing
            df_s["물품명"] = cleansing(list(df_s["물품명"]))
            df_d["물품명"] = cleansing(list(df_d["물품명"]))
            
            ## matching
            df_s["label"] = [match_dic[col["물품명"]]  for row,col in df_s.iterrows()]
            df_d["label"] = [match_dic[col["물품명"]]  for row,col in df_d.iterrows()]

            df_s["bank"] = bank_
            df_s["year"] = year_

            df_d["bank"] = bank_
            df_d["year"] = year_

            df_sall = pd.concat([df_sall,df_s])
            df_dall = pd.concat([df_dall,df_d])

        except:
            error_file.append([year_,bank_p])
    df_sall.to_csv(r"C:\Users\USER\Desktop\All\Project\23_VRP\Data\경기\all_2\{}_supply.txt".format(year_), sep = '\t')
    df_dall.to_csv(r"C:\Users\USER\Desktop\All\Project\23_VRP\Data\경기\all_2\{}_demand.txt".format(year_), sep = '\t')
    print("#"*40)
    

2012


  0%|          | 0/74 [00:00<?, ?it/s]

########################################
2013


  0%|          | 0/74 [00:00<?, ?it/s]

########################################
2014


...

  0%|          | 0/80 [00:00<?, ?it/s]

########################################
2019


  0%|          | 0/91 [00:00<?, ?it/s]

########################################
2020


  0%|          | 0/92 [00:00<?, ?it/s]